In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("Rec-book.csv",engine='python',encoding='ISO-8859-1')

In [3]:
df.head()

,Unnamed: 0,User.ID,Book.Title,Book.Rating
0,1,276726,Classical Mythology,5
1,2,276729,Clara Callan,3
2,3,276729,Decision in Normandy,6
3,4,276736,Flu: The Story of the Great Influenza Pandemic...,8
4,5,276737,The Mummies of Urumchi,6


In [4]:
df.rename({'User.ID':'userID','Book.Title':'Title','Book.Rating':'Rating'},axis=1,inplace=True)

In [5]:
df.head()

,Unnamed: 0,userID,Title,Rating
0,1,276726,Classical Mythology,5
1,2,276729,Clara Callan,3
2,3,276729,Decision in Normandy,6
3,4,276736,Flu: The Story of the Great Influenza Pandemic...,8
4,5,276737,The Mummies of Urumchi,6


In [6]:
df.isnull().sum()

Unnamed: 0    0
userID        0
Title         0
Rating        0
dtype: int64

In [7]:
book= df.drop(['Unnamed: 0'], axis=1)
book

,userID,Title,Rating
0,276726,Classical Mythology,5
1,276729,Clara Callan,3
2,276729,Decision in Normandy,6
3,276736,Flu: The Story of the Great Influenza Pandemic...,8
4,276737,The Mummies of Urumchi,6
...,...,...,...
9995,162121,American Fried: Adventures of a Happy Eater.,7
9996,162121,Cannibal In Manhattan,9
9997,162121,How to Flirt: A Practical Guide,7
9998,162121,Twilight,8


In [8]:
book[book.duplicated()]
bookclean=book[book.duplicated(subset=["userID","Title"])]
bookclean

,userID,Title,Rating
2001,278418,White Oleander : A Novel,5
3792,1211,Black House,9
4745,2033,The First Time,2
5051,2152,Le nouveau soleil de Teur,7
5219,2276,In the Beauty of the Lilies,10
7439,3757,The Magician's Tale,7
9945,162052,Rose,8


In [9]:
len(book['userID'].unique())

2182

In [10]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine,correlation

In [11]:
books_reader=bookclean.pivot(index='userID',columns='Title',values='Rating').reset_index(drop=True)

In [12]:
books_reader

Title,Black House,In the Beauty of the Lilies,Le nouveau soleil de Teur,Rose,The First Time,The Magician's Tale,White Oleander : A Novel
0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2,NaN,NaN,7.0,NaN,NaN,NaN,NaN
3,NaN,10.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,7.0,NaN
5,NaN,NaN,NaN,8.0,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [13]:
books_reader.fillna(0, inplace=True)
books_reader

Title,Black House,In the Beauty of the Lilies,Le nouveau soleil de Teur,Rose,The First Time,The Magician's Tale,White Oleander : A Novel
0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,0.0,0.0,7.0,0.0,0.0,0.0,0.0
3,0.0,10.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,7.0,0.0
5,0.0,0.0,0.0,8.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [14]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine,correlation

In [15]:
reader_sim =1-pairwise_distances(books_reader.values,metric='cosine')
reader_sim

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [16]:
reader_sim_df=pd.DataFrame(reader_sim)

In [17]:
reader_sim_df.index=bookclean.userID.unique()
reader_sim_df.columns=bookclean.userID.unique()

In [18]:
reader_sim_df.iloc[0:5, 0:5]

,278418,1211,2033,2152,2276
278418,1.0,0.0,0.0,0.0,0.0
1211,0.0,1.0,0.0,0.0,0.0
2033,0.0,0.0,1.0,0.0,0.0
2152,0.0,0.0,0.0,1.0,0.0
2276,0.0,0.0,0.0,0.0,1.0


In [19]:
np.fill_diagonal(reader_sim, 0)
reader_sim_df.iloc[0:5, 0:5]

,278418,1211,2033,2152,2276
278418,0.0,0.0,0.0,0.0,0.0
1211,0.0,0.0,0.0,0.0,0.0
2033,0.0,0.0,0.0,0.0,0.0
2152,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0


In [20]:
reader_sim_df.idxmax(axis=1)[0:5]

278418    278418
1211      278418
2033      278418
2152      278418
2276      278418
dtype: int64

In [21]:
bookclean[(bookclean['userID']==278418) |(bookclean['userID']==2276)]

,userID,Title,Rating
2001,278418,White Oleander : A Novel,5
5219,2276,In the Beauty of the Lilies,10


In [22]:
reader_1=bookclean[bookclean['userID']==278418]
reader_1.Title

2001    White Oleander : A Novel
Name: Title, dtype: object

In [23]:
reader_2=bookclean[bookclean['userID']==2276]
reader_2.Title

5219    In the Beauty of the Lilies
Name: Title, dtype: object

In [24]:
pd.merge(reader_1,reader_2,on='Title',how='outer')

,userID_x,Title,Rating_x,userID_y,Rating_y
0,278418.0,White Oleander : A Novel,5.0,NaN,NaN
1,NaN,In the Beauty of the Lilies,NaN,2276.0,10.0
